In [6]:
# テキストコーパスをチャンクに分割
with open('kitei.txt', 'r', encoding='utf-8') as f:
    text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(text)

In [7]:
print(type(texts))
print("--------")
print(len(texts))
print("--------")
print(texts[0])
print("--------")
print(texts[1])

<class 'list'>
--------
19
--------
株式会社ミライソフト 社内規定

■第1章 総則
●第1条（目的）
この規定は、株式会社ミライソフト（以下「当社」という）の円滑な業務運営と、全従業員の安全かつ公正な職場環境の確保を目的とする。
--------
●第2条（適用範囲）
本規定は、当社に雇用される全ての従業員に適用する。


In [8]:
# パッセージのベクトル化
from langchain_huggingface import HuggingFaceEmbeddings

# HuggingFaceのモデルを使用
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large', model_kwargs={'device': 'cpu'})

# OpenAIのモデルを使用
#embeddings = OpenAIEmbeddings()

In [9]:
from langchain_community.vectorstores import FAISS

# データベースの保存
db = FAISS.from_texts(texts, embeddings)
db.save_local('kitei.db')

In [10]:
# 保存したデータベースの読み込み
db = FAISS.load_local('kitei.db',embeddings, allow_dangerous_deserialization=True)

In [11]:
similarity_sample = db.similarity_search("勤務")
print(len(similarity_sample))
print("--------")
print(type(similarity_sample[0]))
print("--------")
print(similarity_sample[0].page_content)

4
--------
<class 'langchain_core.documents.base.Document'>
--------
■第2章 勤務
●第3条（勤務時間）
始業時刻：午前9時00分
終業時刻：午後6時00分
休憩時間：正午12時から午後1時まで（60分）


In [12]:
# 検索器の構築
retriever = db.as_retriever()   # 検索文書数 4（デフォルト）

In [13]:
# モデルの準備
import os
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = 'sk-proj-*****'

openai_llm = ChatOpenAI(model_name="gpt-4.1-nano")
qa = RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    return_source_documents=True,
    )

In [15]:
# 実行例
q = "勤務時間は何時から何時までですか？"
ans = qa.invoke(q)
print(ans['result'])

勤務時間は午前9時00分から午後6時00分までです。
